## Recurrent Neural Network

In [1]:
import torch
from torch import nn, optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

定义超参数

In [2]:
batch_size = 100
learning_rate = 1e-3
num_epoches = 20
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") # whether GPU is supportted

使用手写数字训练集 MNIST

In [3]:
train_dataset = datasets.MNIST('../_data/mnist', train=True, transform=transforms.ToTensor(), download=False)
test_dataset = datasets.MNIST('../_data/mnist', train=False, transform=transforms.ToTensor())

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

定义 RNN 模型

In [5]:
class RNN(nn.Module):
    def __init__(self, in_dim, hidden_dim, n_layer, n_class):
        super(RNN, self).__init__()
        self.n_layer = n_layer
        self.hidden_dim = hidden_dim
        self.lstm = nn.LSTM(in_dim, hidden_dim, n_layer, batch_first=True)
        self.classifier = nn.Linear(hidden_dim, n_class)
        
    def forward(self, x):
        # h0 = torch.zeros(self.n_layer, x.size(1), self.hidden_dim)
        # c0 = torch.zeros(self.n_layer, x.size(1), self.hidden_dim)
        out, _ = self.lstm(x)
        out = out[:, -1, :]
        out = self.classifier(out)
        return out

In [6]:
model = RNN(28, 128, 2, 10).to(device)

定义 loss 和 optimizer

In [7]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

开始训练

In [8]:
for epoch in range(num_epoches):
    print(f'epoch {epoch+1}')
    print('*' * 10)
    running_loss = 0.0
    running_acc = 0.0
    model.train()
    for i, data in enumerate(train_loader, 1):
        img, label = data
        b, c, h, w = img.size()
        assert c == 1, 'channel must be 1'
        img = img.squeeze(1)
        # img = img.view(b*h, w)
        # img = torch.transpose(img, 1, 0)
        # img = img.contiguous().view(w, b, -1) # now have reshape() function
        img, label = img.to(device), label.to(device)
        # forward propagation
        out = model(img)
        loss = criterion(out, label)
        running_loss += loss * label.size(0)
        _, pred = torch.max(out, 1)
        num_correct = (pred == label).sum().float()
        running_acc += num_correct
        # backward propagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if i % 300 == 0:
            print(f'[{epoch+1}/{num_epoches}] Loss: {running_loss/(batch_size * i):.6f}, Acc: {running_acc/(batch_size * i):.6f}')
    print(f'Finish {epoch+1} epoch, Loss: {running_loss/(len(train_dataset)):.6f}, Acc: {running_acc/(len(train_dataset)):.6f}')
    model.eval()
    eval_loss = 0.
    eval_acc = 0.
    for data in test_loader:
        img, label = data
        b, c, h, w = img.size()
        assert c == 1, 'channel must be 1'
        img = img.squeeze()
        # img = img.view(b*h, w)
        # img = torch.transpose(img, 1, 0)
        # img = img.contiguous().view(w, b, h)
        img, label = img.to(device), label.to(device)
        out = model(img)
        loss = criterion(out, label)
        eval_loss += loss * label.size(0)
        _, pred = torch.max(out, 1)
        num_correct = (pred == label).sum().float()
        eval_acc += num_correct
    print(f'Test Loss: {eval_loss/(len(test_dataset)):.6f}, Acc: {eval_acc/(len(test_dataset)):.6f}')

epoch 1
**********
[1/20] Loss: 0.744422, Acc: 0.743600
[1/20] Loss: 0.479028, Acc: 0.839583
Finish 1 epoch, Loss: 0.479028, Acc: 0.839583
Test Loss: 0.169761, Acc: 0.948400
epoch 2
**********
[2/20] Loss: 0.131118, Acc: 0.960300
[2/20] Loss: 0.120595, Acc: 0.963417
Finish 2 epoch, Loss: 0.120595, Acc: 0.963417
Test Loss: 0.086032, Acc: 0.973100
epoch 3
**********
[3/20] Loss: 0.087747, Acc: 0.973400
[3/20] Loss: 0.080368, Acc: 0.975500
Finish 3 epoch, Loss: 0.080368, Acc: 0.975500
Test Loss: 0.072342, Acc: 0.978600
epoch 4
**********
[4/20] Loss: 0.060698, Acc: 0.981333
[4/20] Loss: 0.061054, Acc: 0.981267
Finish 4 epoch, Loss: 0.061054, Acc: 0.981267
Test Loss: 0.061365, Acc: 0.983000
epoch 5
**********
[5/20] Loss: 0.047765, Acc: 0.985667
[5/20] Loss: 0.048526, Acc: 0.985083
Finish 5 epoch, Loss: 0.048526, Acc: 0.985083
Test Loss: 0.049405, Acc: 0.985300
epoch 6
**********
[6/20] Loss: 0.040258, Acc: 0.987333
[6/20] Loss: 0.040447, Acc: 0.987183
Finish 6 epoch, Loss: 0.040447, Acc: 

保存模型

In [9]:
torch.save(model.state_dict(), 'save/05-recurrent neural network.pytorch')